In [37]:
import os
from datetime import datetime, timedelta
from typing import List, Tuple

import pandas as pd
import plotly.graph_objects as go

import tinvest as ti
import requests

In [38]:
api_url_base = 'https://api-invest.tinkoff.ru/openapi/sandbox'
api_token = 't.GsFcZolOZ11aQFAnQud18tTrQ4e7HuH4MZYfDNoV3CWOnzssdwpMwcf0rrhHxkw-AKoWXyo1dAag72CuZ3KFow'
api_headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer {0}'.format(api_token)}

def get_market_search_by_ticker(ticker):
    api_url = '{0}/market/search/by-ticker'.format(api_url_base)
    params = {'ticker': '{0}'.format(ticker)}
    return requests.get(api_url, headers=api_headers, params=params)

get_market_search_by_ticker('AAPL')

<Response [200]>

In [26]:
df = get_market_search_by_ticker('AAPL')

In [28]:
type(df)

requests.models.Response

In [39]:
import asyncio
import tinvest as ti

In [40]:
SANDBOX_TOKEN = api_token
TOKEN = api_token

In [41]:
client = ti.AsyncClient(SANDBOX_TOKEN, use_sandbox=True)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000002A327DF1D90>


In [43]:

def main() -> None:
    payload = client.get_portfolio().payload
    figis = [(p.figi, p.name) for p in payload.positions]
    fig = get_figure(figis)
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()


In [44]:
def get_figure(figis: List[Tuple[str, str]]) -> go.Figure:
    return go.Figure(
        data=[get_candlesstick(get_figi_data(figi), figi, name) for figi, name in figis]
    )

In [45]:

def get_candlesstick(df: pd.DataFrame, figi: str, name: str) -> go.Candlestick:
    return go.Candlestick(
        name=f'{name} {figi}',
        x=df['time'],
        open=df['o'],
        high=df['h'],
        low=df['l'],
        close=df['c'],
    )

In [46]:
def get_figi_data(figi: str) -> pd.DataFrame:
    now = datetime.now()
    payload = client.get_market_candles(
        figi=figi,
        from_=now - timedelta(days=31 * 12),
        to=now,
        interval=ti.CandleResolution.week,
    ).payload
    return pd.DataFrame(c.dict() for c in payload.candles)

In [52]:
client = ti.SyncClient(os.getenv('TINVEST_TOKEN',SANDBOX_TOKEN))

In [53]:

#if __name__ == '__main__':
#    main()
main()

UnexpectedError: 500 {"trackingId":"b724293542927231","payload":{"message":"Invalid token scopes","code":"GATEWAY_REQUEST_DATA_ERROR"},"status":"Error"}

In [42]:

#client = ti.SyncClient(os.getenv('TINVEST_TOKEN', api_token))







AttributeError: 'coroutine' object has no attribute 'payload'

In [36]:
from openapi_client import openapi
from datetime import datetime, timedelta
from pytz import timezone

token = api_token
client = openapi.sandbox_api_client(token)
client.sandbox.sandbox_register_post()
client.sandbox.sandbox_clear_post()
client.sandbox.sandbox_currencies_balance_post(sandbox_set_currency_balance_request={"currency": "USD", "balance": 1000})



ModuleNotFoundError: No module named 'openapi_client'

In [ ]:

def set_balance():
    balance_set = client.sandbox.sandbox_currencies_balance_post({"currency": "USD", "balance": 10000})
    print("balance")
    print(balance_set)
    print()


def print_24hr_operations():
    now = datetime.now(tz=timezone('Europe/Moscow'))
    yesterday = now - timedelta(days=1)
    ops = client.operations.operations_get(_from=yesterday.isoformat(), to=now.isoformat())
    print("operations")
    print(ops)
    print()


def print_orders():
    orders = client.orders.orders_get()
    print("active orders")
    print(orders)
    print()


def make_order():
    order_response = client.orders.orders_limit_order_post(figi='BBG009S39JX6',
                                                           limit_order_request={"lots": 1,
                                                                                "operation": "Buy",
                                                                                "price": 0.01})
    print("make order")
    print(order_response)
    print()
    return order_response


# won't work in sandbox - orders are being instantly executed
def cancel_order(order_id):
    cancellation_result = client.orders.orders_cancel_post(order_id=order_id)
    print("cancel order")
    print(cancellation_result)
    print()


set_balance()
print_24hr_operations()
print_orders()
order_response = make_order()
print_orders()
# cancel_order(order_response.payload.order_id)
# print_orders()